In [10]:
import pandas as pd
import matplotlib.pyplot as plt



In [11]:
games = pd.read_csv('/Users/thanhnguyen/Desktop/Data_analysis/DE_k2/project1_baseball/games/games.csv')
games.head()

,type,multi2,multi3,multi4,multi5,multi6,event,game_id,year
0,id,ALS193307060,NaN,NaN,NaN,NaN,NaN,ALS193307060,1933
1,version,1,NaN,NaN,NaN,NaN,NaN,ALS193307060,1933
2,info,inputprogvers,version 7RS(19) of 07/07/92,NaN,NaN,NaN,NaN,ALS193307060,1933
3,info,visteam,NLS,NaN,NaN,NaN,NaN,ALS193307060,1933
4,info,hometeam,ALS,NaN,NaN,NaN,NaN,ALS193307060,1933


In [12]:
games = games.fillna('')
games.head()

,type,multi2,multi3,multi4,multi5,multi6,event,game_id,year
0,id,ALS193307060,,,,,,ALS193307060,1933
1,version,1,,,,,,ALS193307060,1933
2,info,inputprogvers,version 7RS(19) of 07/07/92,,,,,ALS193307060,1933
3,info,visteam,NLS,,,,,ALS193307060,1933
4,info,hometeam,ALS,,,,,ALS193307060,1933


In [13]:
games.event.unique()

array(['', '63', '63/G6D', ..., 'HR/78/L.2-H;1-H(UR);B-H(UR)',
       'HR/9/L.1-H', 'HR/89/L'], dtype=object)

In [20]:
plays_frame = games.query("type == 'play' & event != 'NP'")
plays_frame.columns = ['type', 'inning', 'team', 'player', 'count', 'pitches', 'event', 'game_id', 'year']
plays_frame.head()

,type,inning,team,player,count,pitches,event,game_id,year
53,play,1,0,martp103,,,63,ALS193307060,1933
54,play,1,0,frisf101,,,63/G6D,ALS193307060,1933
55,play,1,0,kleic101,,,6/L,ALS193307060,1933
56,play,1,1,chapb102,,,53,ALS193307060,1933
57,play,1,1,gehrc101,,,W,ALS193307060,1933


In [24]:
info = games.query("type == 'info' & (multi2 == 'visteam' | multi2 == 'hometeam')")
info = info.loc[:, ['year', 'game_id', 'multi2', 'multi3']]
info.columns = ['year', 'game_id', 'team', 'defense']
info.loc[info['team'] == 'visteam', 'team'] = '1'
info.loc[info['team'] == 'hometeam', 'team'] = '0'
info = info.sort_values(['year', 'game_id', 'team']).reset_index(drop=True)
info.head()

,year,game_id,team,defense
0,1933,ALS193307060,0,ALS
1,1933,ALS193307060,1,NLS
2,1934,NLS193407100,0,NLS
3,1934,NLS193407100,1,ALS
4,1935,ALS193507080,0,ALS


In [28]:
# filter plays_frame for non-pitcher/batter field events
events = plays_frame[~((plays_frame['event'].str.contains('^\d+')) & (~plays_frame['event'].str.contains('E')))]
events = events[~events['event'].str.contains('^(?:P|C|F|I|O)')]

# drop unnecessary columns and sort by team and inning
events = events.drop(['type', 'player', 'count', 'pitches'], axis=1)
events = events.sort_values(['team', 'inning']).reset_index(drop=True)
events.head()

,inning,team,event,game_id,year
0,1,0,S8.B-2(E8),NLS193407100,1934
1,1,0,W,NLS193407100,1934
2,1,0,K,NLS193407100,1934
3,1,0,K+SB2;SB3,NLS193407100,1934
4,1,0,K,NLS193407100,1934


In [30]:
replacements = {
  r'^(?:S|D|T).*': 'H',
  r'^HR.*': 'HR',
  r'^W.*': 'BB',
  r'.*K.*': 'SO',
  r'^HP.*': 'HBP',
  r'.*E.*\..*B-.*': 'RO',
  r'.*E.*': 'E',
}
event_type = events['event'].replace(replacements, regex=True)
events = events.assign(event_type=event_type)

In [31]:
events = events.groupby(['year', 'game_id', 'team', 'event_type']).size().reset_index(name='count')

In [32]:
events

,year,game_id,team,event_type,count
0,1933,ALS193307060,0,H,7
1,1933,ALS193307060,0,HR,1
2,1933,ALS193307060,0,SO,4
3,1933,ALS193307060,1,BB,6
4,1933,ALS193307060,1,H,8
...,...,...,...,...,...
736,2018,NLS201807170,1,BB,6
737,2018,NLS201807170,1,H,2
738,2018,NLS201807170,1,HBP,1
739,2018,NLS201807170,1,HR,5


In [35]:
info.to_csv('/Users/thanhnguyen/Desktop/Data_analysis/DE_k2/project1_baseball/games/info.csv', index=False)
events.to_csv('/Users/thanhnguyen/Desktop/Data_analysis/DE_k2/project1_baseball/games/events.csv', index=False)